# 🛡️ Interactive Security Audit Dashboard
Use the controls below to simulate different model states. The dashboard will automatically calculate security metrics and determine if the model is ready for deployment based on the **Acceptance Criteria Framework**.

### Metrics Explained:
* **Robustness Gap:** The drop in accuracy when under attack (Lower is better).
* **Privacy Budget ($\\epsilon$):** Quantifies privacy loss (Lower is stronger).
* **Extraction Cost:** The estimated financial cost for an attacker to steal the model.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML

# --- 1. Widget Setup ---
style = {'description_width': 'initial'}

# Robustness Sliders
clean_acc_slider = widgets.FloatSlider(value=0.95, min=0.5, max=1.0, step=0.01, description='Clean Accuracy', style=style)
robust_acc_slider = widgets.FloatSlider(value=0.80, min=0.0, max=1.0, step=0.01, description='Robust Accuracy (Under Attack)', style=style)

# Privacy Slider
epsilon_slider = widgets.FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1, description='Privacy Epsilon (ε)', style=style)

# Extraction Inputs
queries_slider = widgets.IntSlider(value=5000, min=100, max=50000, step=100, description='Queries Required', style=style)
cost_per_query = 0.01 # $0.01 per API call

# Output Area
out = widgets.Output()

# --- 2. Logic & Update Function ---
def update_dashboard(change=None):
    clean = clean_acc_slider.value
    robust = robust_acc_slider.value
    eps = epsilon_slider.value
    queries = queries_slider.value
    
    # Calculations
    gap = clean - robust
    extraction_cost = queries * cost_per_query
    
    # Pass/Fail Logic (Thresholds defined in video)
    robust_pass = robust >= 0.85
    privacy_pass = eps <= 1.0
    extraction_pass = extraction_cost > 50.00 # Arbitrary low threshold for demo, usually higher
    
    overall_status = "✅ READY FOR DEPLOYMENT" if (robust_pass and privacy_pass and extraction_pass) else "❌ SECURITY RISKS DETECTED"
    color = "green" if (robust_pass and privacy_pass and extraction_pass) else "red"

    # HTML Template for Dashboard
    html_report = f"""
    <div style='border: 2px solid {color}; padding: 15px; border-radius: 10px; background-color: rgba(200, 200, 200, 0.1);'>
        <h2 style='color: {color}; margin-top: 0;'>{overall_status}</h2>
        <hr>
        
        <h4>1. Robustness Analysis</h4>
        <ul>
            <li><b>Robustness Gap:</b> {gap:.1%} <span style='color: gray'>(Clean - Robust)</span></li>
            <li>Status: {'✅ Pass' if robust_pass else '❌ Fail (Needs > 85% Robust Acc)'}</li>
        </ul>

        <h4>2. Privacy Analysis (Differential Privacy)</h4>
        <ul>
            <li><b>Epsilon (ε):</b> {eps}</li>
            <li>Status: {'✅ Strong Privacy' if privacy_pass else '❌ Weak Privacy (Needs ε <= 1.0)'}</li>
        </ul>

        <h4>3. Extraction Resistance</h4>
        <ul>
            <li><b>Estimated Attack Cost:</b> ${extraction_cost:,.2f}</li>
            <li>Status: {'✅ Expensive to Steal' if extraction_pass else '❌ Cheap to Steal (Needs Cost > $50)'}</li>
        </ul>
    </div>
    """
    
    with out:
        out.clear_output()
        display(HTML(html_report))

# Link sliders to function
clean_acc_slider.observe(update_dashboard, names='value')
robust_acc_slider.observe(update_dashboard, names='value')
epsilon_slider.observe(update_dashboard, names='value')
queries_slider.observe(update_dashboard, names='value')

# --- 3. Display Layout ---
ui = widgets.VBox([
    widgets.HTML("<h3>🎛️ Simulation Controls</h3>"),
    clean_acc_slider, 
    robust_acc_slider, 
    widgets.HTML("<hr>"),
    epsilon_slider, 
    widgets.HTML("<hr>"),
    queries_slider
])

display(widgets.HBox([ui, out]))

# Initialize
update_dashboard()